<a href="https://colab.research.google.com/github/musicjae/recommender-system/blob/main/LBS/location_based_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2

In [17]:
laundry=pd.read_csv('/content/drive/MyDrive/dataset/location-based-recsys/laundry.csv')
user=pd.read_csv('/content/drive/MyDrive/dataset/location-based-recsys/user.csv')

## 데이터 조사

In [5]:
laundry.head(3)

,Unnamed: 0,Count,placeID,ave_rating,latitude,longitude,Service
0,0,56,135032,1.3,22.152481,-100.973486,Cafeteria
1,1,56,135032,1.3,22.152481,-100.973486,Contemporary
2,2,56,135032,1.3,22.152481,-100.973486,Cafeteria


In [9]:
laundry_loc = laundry[['latitude','longitude']]
print(laundry_loc.head(2))

    latitude   longitude
0  22.152481 -100.973486
1  22.152481 -100.973486


In [10]:
user.head(3)

,Unnamed: 0,userID,latitude,longitude,Service
0,0,U1001,22.139997,-100.978803,American
1,1,U1002,22.150087,-100.983325,Mexican
2,2,U1003,22.119847,-100.946527,Mexican


In [12]:
user_loc = user[['latitude','longitude']]
print(user_loc.head(2))

    latitude   longitude
0  22.139997 -100.978803
1  22.150087 -100.983325


In [18]:
laundry.columns

Index(['Unnamed: 0', 'Count', 'placeID', 'ave_rating', 'latitude', 'longitude',
       'Service'],
      dtype='object')

## 데이터 전처리 및 분석

In [19]:
laundry.drop('Unnamed: 0',axis=1,inplace=True)
user.drop('Unnamed: 0',axis=1,inplace=True)

In [21]:
laundry=laundry.groupby(['placeID','latitude','longitude','Service']).mean()
print(laundry.head(2))
laundry.reset_index(inplace=True)
print('\n\n')
print(laundry.head(2))

                                         Count  ave_rating
placeID latitude  longitude   Service                     
132560  23.752304 -99.166913  Regional       4       0.175
132572  22.141647 -100.992712 Cafeteria     15       0.740



   placeID   latitude   longitude    Service  Count  ave_rating
0   132560  23.752304  -99.166913   Regional      4       0.175
1   132572  22.141647 -100.992712  Cafeteria     15       0.740


- 아래는 placeID의 134986이 8번 평가되었음을 의미.  
- 또한 이 placeID는 가장 높은 평가점수를 가졌음을 의미

In [34]:
laundry=laundry.sort_values(by=['ave_rating','Count'], ascending=[False, False])
print(laundry.head(10))
print('\n')
print(laundry.size)

     placeID   latitude   longitude          Service  Count  ave_rating
59    134986  18.928798  -99.239513    International      8    2.700000
91    135055  22.148854 -101.008472          Mexican      7    2.314286
73    135034  22.140517 -101.021422         Japanese      5    2.140000
52    132955  22.147622 -101.010275  Bar_Pub_Brewery      5    2.140000
22    132755  22.153324 -101.019546          Mexican      5    2.020000
102   135074  22.149689 -100.999525     Contemporary      4    1.850000
47    132922  22.151135 -100.982311        Cafeteria      6    1.833333
60    134999  18.915421  -99.184871         Japanese      5    1.740000
25    132768  18.925773  -99.232636           Family     10    1.710000
64    135025  22.149550 -100.977970          Mexican     15    1.706667


672


In [28]:
users = user['userID'].unique()
print(f'유저 수: {len(users)}') #unique users

유저 수: 138


In [29]:
user.head(3)   LBS/location_based_recommender_system.ipynb

,userID,latitude,longitude,Service
0,U1001,22.139997,-100.978803,American
1,U1002,22.150087,-100.983325,Mexican
2,U1003,22.119847,-100.946527,Mexican


## 간단한 추천 모델

- Haversine 공식:  
  https://ttarnawski.usermd.net/2017/08/17/haversine-formula/  
    
    - atan2: https://en.wikipedia.org/wiki/Atan2

In [46]:
class recomendation_py():
    def __init__(self):
        self.user_ID=None
        
    def recomendme(self,user_ID,user,laundry):
        print(user_ID)
        #print(user.head(15))
        #print(laundry.head())
        #print(user.loc[user['userID'] == 'user_ID'])
        #select=user.userID == user_ID
        lat=user.loc[user.userID == user_ID,'latitude'].item()
        long=user.loc[user.userID == user_ID,'longitude'].item()
        serv=user.loc[user.userID == user_ID,'Service'].item()
        print(f'\nuser s latitude: {lat}')
        print(f'user s longitude: {long}')
        print(f'user s service country: {serv}\n')
        laundry=pd.DataFrame(laundry[laundry['Service'] == serv])
        #print(laundry)
        laundry =laundry.reset_index(drop=True)
        R = 6373.0
        list=[]
        for lat1,long1 in zip(laundry['latitude'],laundry['longitude']):
            dlon = long1 - long # 모든 세탁소를 조사하는 동안에, 각각의 세탁소와 입력된 해당 세탁소의 경도를 구한다
            dlat = lat1 - lat #  # 모든 세탁소를 조사하는 동안에, 각각의 세탁소와 입력된 해당 세탁소의 위도를 구한다
            
            a = (sin(dlat/2)**2 + cos(lat)*cos(lat1)*(sin(dlon/2)**2))
            c = 2*atan2(np.sqrt(a),np.sqrt(1-a))
            distance = R*c
            list.append(distance)
        laundry['distance']=pd.Series(list)
        laundry=laundry.sort_values(by=['distance'])

        print(laundry)

In [48]:
pm=recomendation_py()
user_ID = users[21]

pm.recomendme(user_ID,user,laundry)


U1022

user s latitude: 22.146708
user s longitude: -100.964355
user s service country: Mexican

    placeID   latitude   longitude  Service  Count  ave_rating     distance
21   135027  22.147145 -100.974494  Mexican      8    0.425000    63.894140
5    135106  22.149709 -100.976093  Mexican     10    1.410000    76.319124
14   132925  22.153500 -100.976243  Mexican      5    0.880000    86.426499
2    135025  22.149550 -100.977970  Mexican     15    1.706667    87.594419
12   132825  22.147392 -100.983092  Mexican     32    1.009375   118.043198
7    135028  22.146658 -100.987219  Mexican     15    1.293333   143.953640
17   132834  22.156469 -100.985540  Mexican     25    0.724000   147.080130
4    132754  22.147738 -100.990616  Mexican     13    1.423077   165.459093
0    135055  22.148854 -101.008472  Mexican      7    2.314286   278.051714
8    132723  22.148934 -101.019845  Mexican     12    1.225000   349.593117
1    132755  22.153324 -101.019546  Mexican      5    2.020000   34